In [24]:
#find snps
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/permutations/results.csv
#read in data
root = "/Users/bencekover/Library/CloudStorage/OneDrive-Personal/MSci Bahler lab/S.-Pombe-MLPs - Github/"
snps = pd.read_csv(root + 'Bence folder/QTL mapping/permutations_emm_p/results_emm_p.csv')
#washing phenotype /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/washing_phenotypes.csv
wash_pheno = pd.read_csv(root + 'Bence folder/QTL mapping/washing_phenotypes.csv')
#import rna seq corrs /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Analysis of Clement-Ziza RNA-seq/wash_corr.csv
wash_corr = pd.read_csv(root + 'Bence folder/Analysis of Clement-Ziza RNA-seq/wash_corr.csv')
#full snp dataset /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/QTL mapping/SupplementaryDataset_S7_genotype.tsv
full_snps = pd.read_csv(root + 'internal data/updated_genotype_matrix_final_pos.tsv', sep='\t', header=0)
#load orthologs database /Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/Bence folder/Orthologs/final_orthology_database.csv
orthologs = pd.read_csv(root + 'Bence folder/Orthologs/final_orthology_database.csv')
full_snps_og = full_snps.copy()


In [25]:

full_snps.rename(columns={'968':'X968'}, inplace=True)
full_snps.rename(columns={'968_allele':'X968_allele'}, inplace=True)

In [26]:
#iterate through snps and take a given row and the next one. If the Unnamed: 0 is different by less then 500 fill in the gap from full_snps

snps = snps.rename(columns={'Unnamed: 0': 'number'})
snps['number'] = snps['number'] - 1

#create snpsfinal dataframe
snpsfinal = pd.DataFrame()
for i,j in enumerate(snps.number):
    if i == len(snps.number) - 1:
        break
    if (j != snps.number[i+1] - 1) & (snps.number[i+1] - j < 100):
        indices_between = list(range(j+1, snps.number[i+1]))
        #append rows from full_snps to snps to the right index, not end of dataframe
        if snpsfinal.empty:
            snpsfinal = pd.concat([snps.iloc[:i+1], full_snps.iloc[indices_between], snps.iloc[i+1:]], ignore_index=True)
        else:
            snpsfinal = pd.concat([snpsfinal.iloc[:i+1], full_snps.iloc[indices_between], snpsfinal.iloc[i+1:]], ignore_index=True)

snps = snpsfinal.copy()

In [27]:
#remove first 4 columns
full_snps = full_snps.iloc[:,4:]

#re order wash_pheno such that order of strains matches columns in full_snps column names
pca_pheno = pd.DataFrame(columns = ['strain','ratio'])
for i in full_snps.columns:
    #see what is the column name, and find the row in wash_pheno where strain has the same name
    
    row = wash_pheno.loc[wash_pheno['strain'] == i]
    pca_pheno = pca_pheno.append(row)
#pcapheno remove column Unnamed: 0
pca_pheno.drop(columns=['Unnamed: 0'], inplace=True)
#reset index
pca_pheno.reset_index(inplace=True)
#remove chromosome_ from chromosome column
snps['chromosome'] = snps['chromosome'].str.replace('chromosome_', '')
#change chromosome number to roman numbers
snps['chromosome'] = snps['chromosome'].str.replace('1','I')
snps['chromosome'] = snps['chromosome'].str.replace('2','II')
snps['chromosome'] = snps['chromosome'].str.replace('3','III')


In [7]:
snps_t = snps.transpose()
#remove first 5 rows
snps_t = snps_t.iloc[5:]
#by matching index of snps_t to column strain in wash_pheno, we can add the phenotype to the snps_t dataframe
snps_t['phenotype'] = wash_pheno.set_index('strain')['ratio']
#rename all but last column to snps df position 
#snps_t all but last columns rename as snps["position"]
ncols = len(snps_t.columns)
snps_t.columns = np.append(snps["position"].values, snps_t.columns[ncols-1]).flatten()

In [8]:
#for each column perform a t-test between phenotypes of rows with 0 and 1
#store p-values in a list
import scipy.stats as st

p_values = []
#iterate through columns
for i in range(0, len(snps_t.columns)-1):
    #get column
    col = snps_t.iloc[:,i]
    #get phenotype column
    pheno = snps_t.iloc[:,-1]
    #get rows with 0 and 1
    col_0 = col[col == 0]
    col_1 = col[col == 1]
    #get phenotype rows with 0 and 1
    pheno_0 = pheno[col == 0]
    pheno_1 = pheno[col == 1]
    #perform t-test
    t, p = st.ttest_ind(pheno_0, pheno_1)
    p_values.append(p)
#add final entry NA
p_values.append("NA")
#add row with index p_vals
snps_t.loc['p_vals'] = p_values
snps_t



,2604820,2605777,2611769,2612379,2613145,2616395,2617567,2617646,2622695,2624384,...,4582389,4583262,4584481,4585323,4585663,4587327,4588747,4595875,4602196,phenotype
X968,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.04793
Y0036,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.172119
R1_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054247
R1_2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.061369
R1_3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.033055
R1_5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.061934
R1_6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.036447
R1_7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.020331
R1_8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04712
R1_10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028066


In [9]:
#create PCA plot
#take entries except last column and last row
full_snps_pca = full_snps.T
#perform PCA and label with rownames on plot, and colour by respecitve phenotypes (stored in snps_t ["phenotype"])
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
#for NaN values randomly replace them with 0 or 1
for i in range(0, len(full_snps_pca.columns)):
    full_snps_pca.iloc[:,i] = full_snps_pca.iloc[:,i].fillna(np.random.choice([0,1]))
#standardise data
X_std = StandardScaler().fit_transform(full_snps_pca)  
#perform PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X_std)
#add principal components to dataframe
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
#add phenotype column
principalDf['phenotype'] = snps_t.iloc[:-1,-1].values
#change phenotype column to float
principalDf['phenotype'] = pca_pheno["ratio"].astype(float)

#plot
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
#plot each point and color by viridis based on phenotype value
ax.scatter(principalDf['principal component 1'], principalDf['principal component 2'], c=principalDf['phenotype'], cmap='viridis')
#add rownames for points with phenotype above 0.2 or with names X968 and Y0036. Use snps_t_pca.index
for i in range(0, len(principalDf.index)):
    if principalDf.iloc[i,2] > 0.2 or principalDf.index[i] == 'X968' or principalDf.index[i] == 'Y0036':
        ax.annotate(pca_pheno["strain"][i], (principalDf.iloc[i,0], principalDf.iloc[i,1]))
#add variance explained by each component
print(pca.explained_variance_ratio_)
plt.show()

ValueError: could not convert string to float: 'chromosome_1'

In [10]:
#create a new df with 2 columns: phenotype and colum 2528629 if 0 say Ref. allele, if 1 say Mutant
df_plot = pd.DataFrame(columns = ['phenotype','allele'])
df_plot['phenotype'] = snps_t.iloc[:,-1]
 #which column is called 2528629
col_index = snps_t.columns.get_loc("2316554")
df_plot['allele'] = snps_t.iloc[:,col_index]
df_plot['allele'] = df_plot['allele'].replace(0, 'Ref. allele')
df_plot['allele'] = df_plot['allele'].replace(1, 'Mutant')
#keep only rows with Ref. allele or Mutant
df_plot = df_plot[(df_plot['allele'] == 'Ref. allele') | (df_plot['allele'] == 'Mutant')]

#PLOT data now with matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#keep only rows with Ref. allele or Mutant
df_plot = df_plot[(df_plot['allele'] == 'Ref. allele') | (df_plot['allele'] == 'Mutant')]
#create boxplot with outliers as circles and overlay data with scatterplot
sns.boxplot(x="allele", y="phenotype", data=df_plot, whis=np.inf)
#create a jitter of the data and plot that on a scatter plot by adding jitter the Ref. allele and Mutant columns
df_plot['x_coords'] = df_plot['allele'].apply(lambda x: np.random.normal(0, 0.05) if x == 'Ref. allele' else np.random.normal(1, 0.05))
sns.scatterplot(x="x_coords", y="phenotype", data=df_plot, color="black", alpha=0.5)
#plot datapoints with red if their index is in R1_34, R1_44, R1_45, R1_48, R1_49, R1_50, R1_51, R1_52, R1_53, R1_54
for i in range(0, len(df_plot.index)):
      if df_plot.index[i] in ['R1_34', 'R1_44', 'R1_45', 'R1_48', 'R1_49', 'R1_50', 'R1_51', 'R1_52', 'R1_53', 'R1_54']:
            sns.scatterplot(x="x_coords", y="phenotype", data=df_plot.iloc[i:i+1], color="red", alpha=0.5, s=100)
      


plt.title("Washing phenotype: II:2316554 TA vs. A")
plt.xlabel("Allele")
plt.ylabel("Washing phenotype")
#also label each point with strain name

plt.show(p)

KeyError: '2316554'

In [28]:
#create a new df with 2 columns: phenotype and colum 2528629 if 0 say Ref. allele, if 1 say Mutant
df_plot = pd.DataFrame(columns = ['phenotype','allele'])
df_plot['phenotype'] = snps_t.iloc[:,-1]
 #which column is called 2528629
col_index = snps_t.columns.get_loc("2527314")
df_plot['allele'] = snps_t.iloc[:,col_index]
df_plot['allele'] = df_plot['allele'].replace(0, 'Ref. allele')
df_plot['allele'] = df_plot['allele'].replace(1, 'Mutant')
#keep only rows with Ref. allele or Mutant
df_plot = df_plot[(df_plot['allele'] == 'Ref. allele') | (df_plot['allele'] == 'Mutant')]

#PLOT data now with matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
#keep only rows with Ref. allele or Mutant
df_plot = df_plot[(df_plot['allele'] == 'Ref. allele') | (df_plot['allele'] == 'Mutant')]
#create boxplot with outliers as circles and overlay data with scatterplot
sns.boxplot(x="allele", y="phenotype", data=df_plot, whis=np.inf)
#create a jitter of the data and plot that on a scatter plot by adding jitter the Ref. allele and Mutant columns
df_plot['x_coords'] = df_plot['allele'].apply(lambda x: np.random.normal(0, 0.05) if x == 'Ref. allele' else np.random.normal(1, 0.05))
sns.scatterplot(x="x_coords", y="phenotype", data=df_plot, color="black", alpha=0.5)
#plot datapoints with red if their index is in R1_34, R1_44, R1_45, R1_48, R1_49, R1_50, R1_51, R1_52, R1_53, R1_54
for i in range(0, len(df_plot.index)):
      if df_plot.index[i] in ['R1_34', 'R1_44', 'R1_45', 'R1_48', 'R1_49', 'R1_50', 'R1_51', 'R1_52', 'R1_53', 'R1_54']:
            sns.scatterplot(x="x_coords", y="phenotype", data=df_plot.iloc[i:i+1], color="red", alpha=0.5, s=100)
      


plt.title("Washing phenotype: II:2527314 T vs. A")
plt.xlabel("Allele")
plt.ylabel("Washing phenotype")
#also label each point with strain name

plt.show(p)


KeyError: '2527314'

In [29]:
#pandas read in "/Users/bencekover/Library/CloudStorage/OneDrive-UniversityCollegeLondon/MSci Bahler lab/S.-Pombe-biofilm/external data/Pombe genome annotation.gff3"
import pandas as pd
df = pd.read_csv(root + 'external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.gff3', sep='\t', header=1)

#break last column into 2 at ;
df[['a','b']] = df["ID=SPAC1002.01;Name=mrx11"].str.split(';',expand=True)
#remove column ID=SPAC1002.01;Name=mrx11
df.drop(columns =["ID=SPAC1002.01;Name=mrx11"], inplace = True)
#remove ID= from column a and rename it to ID
df['ID'] = df['a'].str.replace('ID=', '')
#remove column a
df.drop(columns =['a','PomBase',".","..1"], inplace = True)
#rename b to Parent
df.rename(columns={'b':'Parent','gene':'object','I':'chromosome','1798347':"start",'1798835':"end","+":"strand"}, inplace=True)
#In parent remove anything before =
df['Parent'] = df['Parent'].str.replace('Parent=', '')
#or Name=
df['Parent'] = df['Parent'].str.replace('Name=', '')
#create an aempty column called detail
df['detail'] = ""
#split ID column at :, and if there is something after put it in detail column
df['detail'] = df['ID'].str.split(':', expand=True)[1]
#split ID column at :, and if there is something before put it in ID column
df['ID'] = df['ID'].str.split(':', expand=True)[0]

In [30]:
#find a subset of df, where snps[position] is +- 1000 of df[start] or df[end]. Create a result df, and say which snps were there
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

result = pd.DataFrame()
for i in range(len(snps)):
    for j in range(len(df)):
        if snps['position'][i] >= df['start'][j] - 500 and snps['position'][i] <= df['end'][j] + 500:
            #and if chromsome is chromosome_2 for df chromsome II etc
            if snps['chromosome'][i] == df['chromosome'][j]:
                result = result.append({'position': snps['position'][i], 'ID': df['ID'][j], 'Parent': df['Parent'][j], 'object': df['object'][j], 'chromosome': df['chromosome'][j], 'start': df['start'][j], 'end': df['end'][j], 'strand': df['strand'][j], 'detail': df['detail'][j]}, ignore_index=True)


#if the last two characters are .1, remove them
for i in range(len(result)):
    if result['ID'][i][-2:] == ".1":
        result['ID'][i] = result['ID'][i][:-2]



In [31]:
#find the p_value of the snp position matches snps_t column_name
#add a column to result called p_value
pvals = np.array([])
for i in range(len(result)):
    for j in range(len(snps_t.columns)):
        #skip if colum nname is phenotype
        if snps_t.columns[j] == 'phenotype':
            continue
        if result['position'][i] == int(snps_t.columns[j]):
            pvals = np.append(pvals, snps_t.iloc[46,j])
result['p_value'] = pvals


In [32]:
#match ID with systematic name in wash_corr and add columns gene	correlation	p-value
#create np.arrays with NAs
corrs = np.array([np.nan]*len(result))
pvals = np.array([np.nan]*len(result))
genes = np.array([np.nan]*len(result), dtype=object)
for i in range(len(result)):
    for j in range(len(wash_corr)):
        if result['ID'][i] == wash_corr['systematic_names'][j]:
            corrs[i] = wash_corr['correlation'][j]
            pvals[i] = wash_corr['p-value'][j]
            genes[i] = wash_corr['gene'][j]

            


result['correlation'] = corrs
result['p_value_corr'] = pvals
result['gene'] = genes

In [33]:


import sys, re

f=open(root + 'external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.fa','r')
lines=f.readlines()

hre=re.compile('>(\S+)')
lre=re.compile('^(\S+)$')

regions={}

for line in lines:
        outh = hre.search(line)
        if outh:
                id=outh.group(1)
        else:
                outl=lre.search(line)
                if(id in regions.keys()):
                        regions[id] += outl.group(1)
                else:
                        regions[id]  =outl.group(1)

In [34]:
#result_final___ only keep rows where snp is between start and end
result_final__ = result.copy()
result_final___ = result_final__.copy()
result_final___ = result_final___[(result_final___['position'] >= result_final___['start']-10) & (result_final___['position'] <= result_final___['end']+100)]

gene_exons = df.copy()
#only keep rows where object is CDS
gene_exons = gene_exons[gene_exons['object'] == 'CDS']
#only keep rows where ID is in result_final__['ID']
gene_exons_final = pd.DataFrame()
for i in np.unique(result_final___['ID']):
      df_subset = gene_exons[gene_exons['ID'] == str(i + ".1")]
      gene_exons_final = gene_exons_final.append(df_subset)
#reset index
gene_exons_final.reset_index(inplace=True)
#make a column exon_number
gene_exons_final['exon_number'] = np.nan
#for those where ID is multiple times, number the exons by increasing start if strand is +, or decreasing start if strand is -
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    if df_subset['strand'].values[0]== '+':
        df_subset = df_subset.sort_values(by=['start'], ascending=True)
    if df_subset['strand'].values[0] == '-':
        df_subset = df_subset.sort_values(by=['end'], ascending=False)
    df_subset['exon_number'] = np.arange(1, len(df_subset) + 1)
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset


#REMOVE entries wehre ID contains SPNCRNA
gene_exons_final = gene_exons_final[~gene_exons_final['ID'].str.contains('SPNCRNA')]





#make a column called real_start
gene_exons_final['old_start'] = gene_exons_final['start']
gene_exons_final['old_end'] = gene_exons_final['end']
#for rows where strand is + look at the load the bases +- 5 around the start from regions
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    df_subset = df_subset[df_subset['exon_number'] == 1]
    if df_subset['strand'].values[0]== '+':
        seq = regions[df_subset['chromosome'].values[0]][int(df_subset['start'].values[0]-5):int(df_subset['start'].values[0]+5)]
        #where does ATG occur in seq
        real_start = seq.find('ATG')-4
        real_start = df_subset['start'] + real_start
        #for the row in gene_exons_final where ID is i and exon_number is 1, change the start to real_start
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == 1), 'start'] = real_start



    else:
        seq_rev = regions[df_subset['chromosome'].values[0]][int(df_subset['end'].values[0]-5):int(df_subset['end'].values[0]+5)]
        real_end = seq_rev.find('CAT')+2-4
        real_end = df_subset['end'] + real_end
        #for the row in gene_exons_final where ID is i and exon_number is 1, change the end to real_end
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == 1), 'end'] = real_end


    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    highest_exon = df_subset['exon_number'].max()
    df_subset = df_subset[df_subset['exon_number'] == highest_exon]
    if df_subset['strand'].values[0]== '+':

        seq2 = regions[df_subset['chromosome'].values[0]][int(df_subset['end'].values[0]-5):int(df_subset['end'].values[0]+5)]
        real_end = seq2.find('TAA')+2-4
        real_end = df_subset['end'] + real_end
        #for the row in gene_exons_final where ID is i and exon_number is highest_exon, change the end to real_end
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == highest_exon), 'end'] = real_end
    else:
        seq2_rev = regions[df_subset['chromosome'].values[0]][int(df_subset['start'].values[0]-5):int(df_subset['start'].values[0]+5)]
        real_start = seq2_rev.find('TCA')+2-4
        real_start = df_subset['start'] + real_start
        #for the row in gene_exons_final where ID is i and exon_number is highest_exon, change the start to real_start
        gene_exons_final.loc[(gene_exons_final['ID'] == i) & (gene_exons_final['exon_number'] == highest_exon), 'start'] = real_start


    
    



#for each row calculate the length of the exon
gene_exons_final['exon_length'] = abs(gene_exons_final['end'] - gene_exons_final['start'])+1

#remove rows where ID is NaN
gene_exons_final = gene_exons_final[~gene_exons_final['ID'].isna()]
#for each row calculate the length of the CDS until that exon starts
gene_exons_final['CDS_length_so_far'] = np.nan
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    #order by exon_number
    df_subset = df_subset.sort_values(by=['exon_number'])
    
    df_subset['CDS_length_so_far'] = np.append(np.array(0),df_subset['exon_length'].cumsum())[:-1]
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset
#add total cds length
gene_exons_final['total_CDS_length'] = np.nan
for i in np.unique(gene_exons_final['ID']):
    df_subset = gene_exons_final[gene_exons_final['ID'] == i]
    df_subset['total_CDS_length'] = sum(df_subset['exon_length'])
    gene_exons_final[gene_exons_final['ID'] == i] = df_subset

gene_exons_final

,index,chromosome,object,start,end,strand,Parent,ID,detail,exon_number,old_start,old_end,exon_length,CDS_length_so_far,total_CDS_length
0,955,I,CDS,4570495,4575441,-,SPAC12B10.01c.1,SPAC12B10.01c.1,exon,1.0,4570498,4575441,4947,0.0,4947.0
1,965,I,CDS,4578627,4580258,+,SPAC12B10.03.1,SPAC12B10.03.1,exon,1.0,4578627,4580258,1632,0.0,1632.0
2,979,I,CDS,4582378,4582549,+,SPAC12B10.05.1,SPAC12B10.05.1,exon,1.0,4582378,4582549,172,0.0,1458.0
3,981,I,CDS,4582595,4582697,+,SPAC12B10.05.1,SPAC12B10.05.1,exon,2.0,4582595,4582697,103,172.0,1458.0
4,983,I,CDS,4582747,4582767,+,SPAC12B10.05.1,SPAC12B10.05.1,exon,3.0,4582747,4582767,21,275.0,1458.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,15957,I,CDS,4532852,4532924,+,SPATRNAASP.01.1,SPATRNAASP.01.1,exon,1.0,4532857,4532927,73,0.0,73.0
353,16038,I,CDS,3086305,3086348,+,SPATRNALYS.05.1,SPATRNALYS.05.1,exon,1.0,3086310,3086348,44,0.0,84.0
354,16040,I,CDS,3086357,3086396,+,SPATRNALYS.05.1,SPATRNALYS.05.1,exon,2.0,3086357,3086392,40,44.0,84.0
355,16096,I,CDS,3164744,3164817,+,SPATRNATHR.04.1,SPATRNATHR.04.1,exon,1.0,3164749,3164820,74,0.0,74.0


In [35]:
#add to each row the unique number of positions that ID had
#add a column called unique
#remove duplicates of ID and position
#keep only entries with "object" is CDS
result = result[result['object'] == 'CDS']
result_final = result.drop_duplicates(subset=['ID', 'position'])
#reset index
result_final = result_final.reset_index(drop=True)
unique = np.array([])
for i in range(len(result_final)):
    #get the ID
    ID = result_final['ID'][i]
    #get the number of times ID appears in result
    count = len(result_final[result_final['ID'] == ID])
    #add count to unique
    unique = np.append(unique, count)
result_final['unique'] = unique



In [36]:
#show SPBC290.07 in result
#gather positions for each ID, and put in a single entry as a list. Add that to the df. THen remove duplicated IDs
result_final_ = result_final.groupby('ID')['position'].apply(list).reset_index(name='positions')
result_final__ = pd.merge(result_final, result_final_, on='ID')

#remove duplicates of ID
result_final__ = result_final__.drop_duplicates(subset=['ID'])

#reset index
result_final__ = result_final__.reset_index(drop=True)

In [37]:
#iterate through result_final__. If ID is found in orthologs, then add the albicans_name(s)cerevisiae_name(s) to the df
from numpy import dtype

albicans_names = np.array([np.nan]*len(result_final__),dtype=object)
cerevisiae_names = np.array([np.nan]*len(result_final__),dtype=object)
for i in range(len(result_final__)):
    for j in range(len(orthologs)):
        if result_final__['ID'][i] in orthologs['pombe_name(s)'][j]:
            albicans_names[i] = orthologs['albicans_name(s)'][j]
            cerevisiae_names[i] = orthologs['cerevisiae_name(s)'][j]
#add to df
result_final__['albicans_name(s)'] = albicans_names
result_final__['cerevisiae_name(s)'] = cerevisiae_names

In [38]:
#sort by p_value
result_final__ = result_final__.sort_values(by=['p_value_corr'])
result_final__.head(50)

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,unique,positions,albicans_name(s),cerevisiae_name(s)
424,4473108,SPAC17C9.16c,SPAC17C9.16c.1,CDS,I,4472611,4473271,+,exon,1.0,0.678393,3.042498e-07,mfs1,6.0,"[4473108, 4473996, 4474485, 4474680, 4474681, ...",NaN,NaN
195,3006775,SPAC31G5.11,SPAC31G5.11.1,CDS,I,3006138,3006845,+,exon,1.0,0.664822,6.329116e-07,pac2,1.0,[3006775],['C1_10150W_A' 'nan'],['YEL007W' 'YHR177W']
194,3002877,SPNCRNA.872,SPNCRNA.872.1,CDS,I,3001750,3004335,-,exon,1.0,0.618445,5.970721e-06,eta2-antisense-1,1.0,[3002877],NaN,NaN
425,4473108,SPNCRNA.991,SPNCRNA.991.1,CDS,I,4472218,4473227,-,exon,1.0,-0.561792,5.931923e-05,mfs1-antisense-1,1.0,[4473108],NaN,NaN
276,3147678,SPAC3G9.14,SPAC3G9.14.1,CDS,I,3147207,3149507,-,exon,1.0,0.464559,1.305065e-03,sak1,2.0,"[3147678, 3149857]",NaN,NaN
454,4531317,SPAC27D7.11c,SPAC27D7.11c.1,CDS,I,4529462,4530853,-,exon,1.0,-0.452833,1.787387e-03,SPAC27D7.11c,1.0,[4531317],NaN,NaN
253,3103928,SPNCRNA.560,SPNCRNA.560.1,CDS,I,3104356,3105524,+,exon,1.0,0.451910,1.831298e-03,atg1801-antisense-1,1.0,[3103928],NaN,NaN
233,3074774,SPAC24C9.15c,SPAC24C9.15c.1,CDS,I,3074772,3075353,-,exon,1.0,-0.438076,2.614700e-03,spn5,2.0,"[3074774, 3075383]",NaN,NaN
91,2759901,SPAC6F6.19,SPAC6F6.19.1,CDS,I,2759479,2759709,-,exon,1.0,0.415810,4.501092e-03,SPAC6F6.19,1.0,[2759901],NaN,NaN
109,2797593,SPAC1805.14,SPAC1805.14.1,CDS,I,2796469,2797827,+,exon,1.0,0.412780,4.833124e-03,SPAC1805.14,1.0,[2797593],NaN,NaN


In [39]:
result_final__ = result.copy().reset_index(drop=True, inplace=False)

In [40]:
#iterate through positions in result_final__  and find the corresponding alleles in snps. Then add the two values in X968_allele	Y0036_allele to the df in two new columns at the start 
#add two columns to result_final__
result_final__['X968_allele'] = np.nan
result_final__['Y0036_allele'] = np.nan
for i in range(len(result_final__)):
    for j in range(len(snps)):
        if result_final__['position'][i] == snps['position'][j]:
            result_final__['X968_allele'][i] = snps['X968_allele'][j]
            result_final__['Y0036_allele'][i] = snps['Y0036_allele'][j]

In [41]:
#iterate through positions in result_final__ and look up the chromosome and position in regions, and find the corresponding letter +/- 5 bp in regions. Add it to a new column
result_final__['reference_seq']=np.nan
for i in range(len(result_final__)):
    for j in range(len(regions)):
        if result_final__['chromosome'][i] == list(regions.keys())[j]:
            #add | | characters left and right of tletter at position-1
            
            result_final__['reference_seq'][i] = regions[list(regions.keys())[j]][result_final__['position'][i]-4:result_final__['position'][i]-1] + ' |' + regions[list(regions.keys())[j]][result_final__['position'][i]-1] + '| ' + regions[list(regions.keys())[j]][result_final__['position'][i]:result_final__['position'][i]+3] 

#if strand is -, reverse reference_seq
for i in range(len(result_final__)):
    if result_final__['strand'][i] == '-':
        result_final__['reference_seq'][i] = result_final__['reference_seq'][i][::-1]
        #also flip the alleles
        result_final__['X968_allele'][i] = result_final__['X968_allele'][i][::-1]
        result_final__['Y0036_allele'][i] = result_final__['Y0036_allele'][i][::-1]
#also create reference_seq_ without the | | characters
result_final__['reference_seq_'] = result_final__['reference_seq'].str.replace('|', '')
result_final__['reference_seq_'] = result_final__['reference_seq_'].str.replace(' ', '')




In [42]:
#create column, whole gene as codons
result_final__['wt_gene'] = np.nan
#iterate through result_final__ positions. If object is CDS, then look up the sequence from start to end in regions.
#add the sequence to the df
for i in range(len(result_final__)):
    
    if result_final__['object'][i] == 'CDS':
    
        start = result_final__['start'][i]
        end = result_final__['end'][i]
        chrom = result_final__['chromosome'][i]
        pos = result_final__['position'][i]
        gene = regions[chrom][start-1:end]
        #gene_exons_final where start is the same as start in result_final__
        exon = gene_exons_final[(gene_exons_final['start'] < start+20) & (gene_exons_final['end'] > end-20)] 
        #if exon is empty, then print gene name
        if exon.empty:
            print(result_final__['ID'][i])
            continue
        exon_len_so_far=0
        exon_len_so_far = exon['CDS_length_so_far'].values[0]
        exon_len_so_far=exon_len_so_far%3
        if pos > start and pos < end:
        
            if result_final__['strand'][i] == '-':
                
                gene = gene[::-1]
            
                gene = ''.join(gene)
            gene = 'A'*int(exon_len_so_far) + gene
            #write the gene as triplets of letters like AAA, ATC, etc
            #if divisible by 3
            if len(gene)%3 != 0:
                gene = gene[:-(len(gene)%3)]
            gene = [gene[i:i+3] for i in range(0, len(gene), 3)]
                #only triplets near the position
            if result_final__['strand'][i] == '+':
                gene = gene[(pos-(start-int(exon_len_so_far)))//3-2:(pos-end)//3+25]
            else: 
                gene = gene[(end-pos)//3-2:(start-int(exon_len_so_far)-pos)//3+25]
            
                
                


            result_final__['wt_gene'][i] = gene

result_final__['mutant allele'] = np.nan
for i in range(len(result_final__)):
    if result_final__['object'][i] == 'CDS':
    
        start = result_final__['start'][i]
        end = result_final__['end'][i]
        chrom = result_final__['chromosome'][i]
        pos = result_final__['position'][i]
        gene = regions[chrom][start-1:end]
         #gene_exons_final where start is the same as start in result_final__
        exon = gene_exons_final[(gene_exons_final['start'] < start+20) & (gene_exons_final['end'] > end-20)] 
        #if exon is empty, then print gene name
        if exon.empty:
            print(result_final__['ID'][i])
            continue
        exon_len_so_far=0
        exon_len_so_far = exon['CDS_length_so_far'].values[0]
        exon_len_so_far=exon_len_so_far%3
        if pos > start and pos < end:
        
            if result_final__['strand'][i] == '+':
                #decide which allele to use
                #check 
                #check how long the y0036 allele is 
                x968_len = len(result_final__['X968_allele'][i])
                y0036_len = len(result_final__['Y0036_allele'][i])
                
                mutation = result_final__['Y0036_allele'][i]
               

                gene = gene[:pos-start] + mutation + gene[pos-start+x968_len:]
            elif result_final__['strand'][i] == '-':
                x968_len = len(result_final__['X968_allele'][i])
                y0036_len = len(result_final__['Y0036_allele'][i])
                gene = gene[::-1]
                gene = ''.join(gene)
                mutation = result_final__['Y0036_allele'][i]
                
                gene = gene[:end-pos] + mutation + gene[end-pos+x968_len:]
                
            gene = 'A'*int(exon_len_so_far) + gene
            if len(gene)%3 != 0:
                gene = gene[:-(len(gene)%3)]
            gene = [gene[k:k+3] for k in range(0, len(gene), 3)]
                #only triplets near the position
            if result_final__['strand'][i] == '+':
                gene = gene[(pos-(start-int(exon_len_so_far)))//3-2:(pos-end)//3+25]
            else:
                gene = gene[(end-pos)//3-2:(start-int(exon_len_so_far)-pos)//3+25]
            result_final__["mutant allele"][i] = gene
                
           


SPNCRNA.3165
SPAC823.14
SPNCRNA.3165
SPNCRNA.3166
SPAC7D4.13c
SPAC7D4.13c
SPNCRNA.3169
SPNCRNA.3171
SPAC7D4.11c
SPNCRNA.3171
SPAC7D4.11c
SPNCRNA.3171
SPNCRNA.3172
SPAC7D4.10
SPNCRNA.3173
SPNCRNA.842
SPAC7D4.15c
SPAC7D4.15c
SPNCRNA.3178
SPNCRNA.3178
SPNCRNA.3184
SPNCRNA.3184
SPAC4F8.12c
SPNCRNA.3189
SPNCRNA.326
SPNCRNA.3189
SPNCRNA.200
SPNCRNA.3196
SPAC4F8.01
SPNCRNA.3199
SPAC644.05c
SPNCRNA.3202
SPNCRNA.844
SPNCRNA.3207
SPNCRNA.3207
SPNCRNA.3208
SPNCRNA.202
SPNCRNA.3212
SPNCRNA.3213
SPNCRNA.3214
SPNCRNA.846
SPNCRNA.846
SPNCRNA.3216
SPNCRNA.3217
SPNCRNA.3218
SPAPB2B4.06
SPNCRNA.3220
SPAPB2B4.07
SPNCRNA.3221
SPNCRNA.204
SPNCRNA.3222
SPNCRNA.847
SPNCRNA.847
SPNCRNA.847
SPNCRNA.848
SPNCRNA.3227
SPNCRNA.3227
SPNCRNA.3227
SPNCRNA.3227
SPNCRNA.3228
SPNCRNA.3230
SPNCRNA.3230
SPAC6F6.10c
SPNCRNA.3235
SPNCRNA.849
SPNCRNA.849
SPNCRNA.849
SPNCRNA.849
SPNCRNA.849
SPNCRNA.850
SPAC6F6.19
SPNCRNA.3239
SPNCRNA.851
SPNCRNA.3239
SPNCRNA.3240
SPNCRNA.3240
SPNCRNA.3242
SPNCRNA.3242
SPNCRNA.852
SPNCRNA.3244

In [43]:
#CreATE A DF WHERE WHOLE GENE IS NOT NAN
stop_codon_df = result_final__[result_final__['wt_gene'].notna()]
stop_codon_df["stop_codon_predicted"] = np.nan
#iterate through mutant allele, and see whether it contains
#reset index
stop_codon_df = stop_codon_df.reset_index(drop=True)

for i in range(len(stop_codon_df)):
    for j in range(len(stop_codon_df['mutant allele'][i])):
        if stop_codon_df["strand"][i] == '+':
            if stop_codon_df['mutant allele'][i][j] == 'TAA' or stop_codon_df['mutant allele'][i][j] == 'TAG' or stop_codon_df['mutant allele'][i][j] == 'TGA':
                stop_codon_df['stop_codon_predicted'][i] = 'yes'
        else:
            if stop_codon_df['mutant allele'][i][j] == 'ATT' or stop_codon_df['mutant allele'][i][j] == 'ATC' or stop_codon_df['mutant allele'][i][j] == 'ACT':
                stop_codon_df['stop_codon_predicted'][i] = 'yes'
#show which entries have a stop codon predicted
stop_codon_df[stop_codon_df['stop_codon_predicted'] == 'yes']


,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,X968_allele,Y0036_allele,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted
3,2612379,SPNCRNA.3168,SPNCRNA.3168.1,CDS,I,2611788,2612409,-,exon,1.0,NaN,NaN,NaN,G,A,TAC |G| GTA,TACGGTA,"[ACT, TAC, GGT, AAT, ATT, CCT, CCT, TCG, CTA, ...","[ACT, TAC, AGT, AAT, ATT, CCT, CCT, TCG, CTA, ...",yes
14,2648315,SPNCRNA.3183,SPNCRNA.3183.1,CDS,I,2647955,2648490,-,exon,1.0,NaN,NaN,NaN,T,C,GTT |T| ACT,GTTTACT,"[TTA, GGT, TTA, CTA, ACA, GCC, GAT, GAT, TTT, ...","[TTA, GGT, TCA, CTA, ACA, GCC, GAT, GAT, TTT, ...",yes
18,2658731,SPNCRNA.3188,SPNCRNA.3188.1,CDS,I,2658334,2658989,+,exon,1.0,NaN,NaN,NaN,G,A,TTC |G| TTA,TTCGTTA,"[ACA, TTT, CGT, TAT, GTG, ACG, TAC, CAG, AAT, ...","[ACA, TTT, CAT, TAT, GTG, ACG, TAC, CAG, AAT, ...",yes
40,2766755,SPNCRNA.3243,SPNCRNA.3243.1,CDS,I,2766609,2767122,-,exon,1.0,NaN,NaN,NaN,T,C,ATC |T| AAA,ATCTAAA,"[ACA, TAT, CTA, AAA, GTT, TTT, CCT, TCT, GTC, ...","[ACA, TAT, CCA, AAA, GTT, TTT, CCT, TCT, GTC, ...",yes
44,2779113,SPNCRNA.3247,SPNCRNA.3247.1,CDS,I,2778732,2779145,-,exon,1.0,NaN,NaN,NaN,T,C,GAT |T| GTC,GATTGTC,"[TTT, ATA, GAT, TGT, CTT, AAC, GTA, AAA, AGC, ...","[TTT, ATA, GAT, CGT, CTT, AAC, GTA, AAA, AGC, ...",yes
51,2905076,SPNCRNA.3311,SPNCRNA.3311.1,CDS,I,2903507,2905392,+,exon,1.0,NaN,NaN,NaN,C,T,AAA |C| AAG,AAACAAG,"[AGC, AAA, CAA, GTC, CTG, CAA, AGA, TAC, ATT, ...","[AGC, AAA, TAA, GTC, CTG, CAA, AGA, TAC, ATT, ...",yes
82,3092232,SPNCRNA.3385,SPNCRNA.3385.1,CDS,I,3091339,3092431,+,exon,1.0,NaN,NaN,NaN,G,A,GAG |G| AGA,GAGGAGA,"[CTG, TAG, AGG, AGA, ATT, AGA, AGT, ATT, ACC, ...","[CTG, TAG, AGA, AGA, ATT, AGA, AGT, ATT, ACC, ...",yes
110,3202214,SPNCRNA.3430,SPNCRNA.3430.1,CDS,I,3201501,3202270,-,exon,1.0,NaN,NaN,NaN,T,C,GAT |T| TTC,GATTTTC,"[ATG, GAA, GAT, TTT, CTA, GGT, TTA, TTA, GCC, ...","[ATG, GAA, GAT, CTT, CTA, GGT, TTA, TTA, GCC, ...",yes
122,3292079,SPNCRNA.3473,SPNCRNA.3473.1,CDS,I,3291576,3292459,+,exon,1.0,NaN,NaN,NaN,T,C,CAC |T| TCC,CACTTCC,"[CTC, ACC, ACT, TCC, GAA, AAT, GTT, CCA, AAC, ...","[CTC, ACC, ACC, TCC, GAA, AAT, GTT, CCA, AAC, ...",yes
149,3402585,SPNCRNA.3518,SPNCRNA.3518.1,CDS,I,3402491,3402911,+,exon,1.0,NaN,NaN,NaN,G,A,CGC |G| CAA,CGCGCAA,"[TTA, CGC, GCA, AGA, TGT, TCT, AGT, ATT, TGA, ...","[TTA, CGC, ACA, AGA, TGT, TCT, AGT, ATT, TGA, ...",yes


In [44]:
#in stop_codon_df create a column "possible frameshift" and fill it with nan. Then loop through X96_allele and Y0036_allele and check if the length is different. If yes and the difference is not divisible by 3, fill the column with "yes". Then show the entries where the column is "yes"
frameshift_df = stop_codon_df.copy()
frameshift_df['possible frameshift'] = np.nan

for i in range(len(frameshift_df)):
    if len(frameshift_df['X968_allele'][i]) != len(frameshift_df['Y0036_allele'][i]):
        if (len(frameshift_df['X968_allele'][i]) - len(frameshift_df['Y0036_allele'][i]))%3 != 0:
            frameshift_df['possible frameshift'][i] = 'yes'

In [45]:
#import a codon table as a dictionary
codon_table = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*',
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q', 
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M',
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}
#iterate through frameshift_df and trnslate the lists of triplets found in wt_gene and mutant allele
frameshift_df['wt_gene_translated'] = np.nan
frameshift_df['mutant allele_translated'] = np.nan
for i in range(len(frameshift_df)):
    #if strand is -, then change a to t and so on
    if frameshift_df['strand'][i] == '-':
        temp_wt_allele = frameshift_df['wt_gene'][i]
        temp_mut_allele = frameshift_df['mutant allele'][i]
        for j in range(len(temp_wt_allele)):
            temp_wt_allele[j] = temp_wt_allele[j].replace('A','t')
            temp_wt_allele[j] = temp_wt_allele[j].replace('T','a')
            temp_wt_allele[j] = temp_wt_allele[j].replace('C','g')
            temp_wt_allele[j] = temp_wt_allele[j].replace('G','c')
            temp_wt_allele[j] = temp_wt_allele[j].upper()
        for j in range(len(temp_mut_allele)):
            temp_mut_allele[j] = temp_mut_allele[j].replace('A','t')
            temp_mut_allele[j] = temp_mut_allele[j].replace('T','a')
            temp_mut_allele[j] = temp_mut_allele[j].replace('C','g')
            temp_mut_allele[j] = temp_mut_allele[j].replace('G','c')
            temp_mut_allele[j] = temp_mut_allele[j].upper()
        frameshift_df['wt_gene_translated'][i] = ''.join([codon_table[codon] for codon in temp_wt_allele])
        frameshift_df['mutant allele_translated'][i] = ''.join([codon_table[codon] for codon in temp_mut_allele])
    else:
        frameshift_df['wt_gene_translated'][i] = ''.join([codon_table[codon] for codon in frameshift_df['wt_gene'][i]])
        frameshift_df['mutant allele_translated'][i] = ''.join([codon_table[codon] for codon in frameshift_df['mutant allele'][i]])

In [46]:
#count how many times wt_gene_translated is not equal to mutant allele_translated when ID doesnt contain SPNC
frameshift_df[(frameshift_df['ID'].str.contains('SPNC') == False) & (frameshift_df['wt_gene_translated'] != frameshift_df['mutant allele_translated'])].shape[0]

91

In [54]:
#show where gene doesnt start with SPNC and possible frameshift
#show all rows and stop_codon_predicted
pd.set_option('display.max_rows', None)
frameshift_df[(frameshift_df['ID'].str.contains('SPNC') == False) & (frameshift_df['stop_codon_predicted'] == 'yes')]

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,X968_allele,Y0036_allele,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted,possible frameshift,wt_gene_translated,mutant allele_translated
171,4473996,SPAC17C9.16c,SPAC17C9.16c.1,CDS,I,4473325,4474259,+,exon,1.0,0.678393,3.042498e-07,mfs1,C,T,TTC |C| AAA,TTCCAAA,"[TTG, TTC, CAA, AGT, ATG, TTT, CAA, TAT, TTG, ...","[TTG, TTC, TAA, AGT, ATG, TTT, CAA, TAT, TTG, ...",yes,NaN,LFQSMFQYLAAAYPKYVASVFAGNALF,LF*SMFQYLAAAYPKYVASVFAGNALF
198,4582389,SPAC12B10.05,SPAC12B10.05.1,CDS,I,4582378,4582549,+,exon,1.0,0.194993,1.992787e-01,icp55,T,A,TTA |T| ATC,TTATATC,"[AGC, GGT, TAT, ATC, CGG, ACT, CTC, TTT, ATT, ...","[AGC, GGT, TAA, ATC, CGG, ACT, CTC, TTT, ATT, ...",yes,NaN,SGYIRTLFIRNRFSNYRLRSQIIKYKY,SG*IRTLFIRNRFSNYRLRSQIIKYKY


In [55]:
#where possible frameshift is yes
frameshift_df[frameshift_df['possible frameshift'] == 'yes']

,position,ID,Parent,object,chromosome,start,end,strand,detail,p_value,correlation,p_value_corr,gene,X968_allele,Y0036_allele,reference_seq,reference_seq_,wt_gene,mutant allele,stop_codon_predicted,possible frameshift,wt_gene_translated,mutant allele_translated
